In [1]:
import pandas as pd
import numpy as np
import zipfile
import pysurveycto
import datetime
import pandas as pd
import pytz
from datetime import datetime, timedelta
from io import StringIO
import re
import pywhatkit as kit
import time
import warnings as wn
wn.filterwarnings("ignore")
import streamlit as st 

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# survey login credentials
server_name = "rtvuganda"
username = "raphael@raisingthevillage.org"
password = "Evaluation2022"

In [ ]:
# passing the credentials to the server
scto = pysurveycto.SurveyCTOObject(server_name, username, password)

In [ ]:
# current survey form id
form_id = "bhs_2025_rtv"

In [ ]:
# form_id = "bhs_pilot_2025_rtv"

In [ ]:
# extracting the survey data from the survey
my_form_data = scto.get_form_data(form_id, format="json", shape="long")

In [ ]:
# converting the extracted data to pandas data frame
data = pd.DataFrame(my_form_data)

In [ ]:
data.to_csv("allbhs2025.csv")

In [ ]:
# converting submissionDate to pandas data time format
data['SubmissionDate'] = pd.to_datetime(data['SubmissionDate'])

In [ ]:
#generating the yesterday date for easy tracking samples of yesterday
yesterday = (datetime.today() - timedelta(days=1)).date()

In [ ]:
# converting the data of status and consent_1 form surverycto datatypes to pandas float datatype
data[['status', 'consent_1']] = data[['status', 'consent_1']].astype(float)

In [ ]:
data['SubmissionDate'].unique

In [ ]:
# filtering the data for the previous day and data for samples fully completed
data = data[
    (data['SubmissionDate'].dt.date == yesterday) &
    ((data['status'] == 1) | (data['status'] == 7)) &
    (data['consent_1'] == 1)
]

In [ ]:
data.shape

In [ ]:
# converting the starttime to datatime in pandas
data['starttime'] = pd.to_datetime(data['starttime'])

In [ ]:
# converting the endtime to datatime in pandas
data['endtime']=pd.to_datetime(data['endtime'])

In [ ]:
#converting duration to numeric
data['duration'] = pd.to_numeric(data['duration'], errors='coerce') 

In [ ]:
# importing excel files with columns to drop to enhance styling process of the dataframe
# please include all columns to drop in this sheet
column_drop_df=pd.read_excel(r"C:\\Users\\Edison New\\Desktop\\edison jupyter\\Copy of qualitychecks_columns_drop.xlsx")

In [ ]:
# creating function to drop to drop columns and  dropping columns from data
def columns_drop(data, column_drop_df):
    columns_to_drop = column_drop_df['Column Names'].to_list()
    data = data.drop(columns=columns_to_drop, errors='ignore')
    return data
# dropping un wanted columns from data
data = columns_drop(data, column_drop_df)

In [ ]:
# converting the duration from seconds to minutes under new variable duration2
data['duration2']=data['duration']/60

In [ ]:
# creating the column check for samples started beyond 8PM
data['is_start_time_beyond_8pm'] = (pd.to_datetime(data['starttime']).dt.hour >= 20).astype(int)


In [ ]:
# checking whether there if date difference between when the sample was started vs endtime of the sample
data['starttime'] = pd.to_datetime(data['starttime']).dt.date
data['endtime'] = pd.to_datetime(data['endtime']).dt.date
data['date_difference'] = (data['endtime'] - data['starttime']).apply(lambda x: x.days)

In [ ]:
# drop
data['is_duration_invalid']=((data['duration2']<20) | (data['duration2']>60)).astype(int)

In [ ]:
# Creating columns for samples collected by enumerator the pervious day
data['sample'] = data['enumerator_name'].map(data['enumerator_name'].value_counts())

In [ ]:
list_ = ['beans', 'maize','peas','cassava']  
season = [1, 2]
state = ['fresh', 'dry']
price_1 = []

for i in season:
    for j in list_:
        for k in state:
            price1 = f'sn_{i}_{j}_Market_Price_{k}'
            if price1 in data.columns:
                price_1.append(price1)  

In [ ]:
list=['gnuts','yams','sweetpotatoes','irish_potatoes',
      'ginger','garlic','rice','sorghum','millet',
      'soya_beans']
season=[1,2]
price_2=[]
for i in list:
    for j in season:
        price1=f'sn_{j}_{i}_Market_Price'
        if price1 in data.columns:
            price_2.append(price1)       

In [ ]:
# merging prices of seasonal crops into one list for ease accessibility
season_crop_prices=price_1+price_2

In [ ]:
#Converting all season crop prices to numeric from json data structures
for column in season_crop_prices:
    data[column] = pd.to_numeric(data[column], errors='coerce')

In [ ]:
# importing the excel sheet with min,max of major seasonal crops,vegetables, perennial crops
price_df=pd.read_excel("crop_prices.xlsx")

In [ ]:
# extracting vegetables prices from the data to lists for easy accessibility
veg = [
    "Pumpkins",
    "Carrots",
    "onions",
    "green_pepper",
    "hot_pepper",
    "cabbage",
    "tomato",
    "watermelon",
    "spinach",
    "cauliflower",
    "sukuma_wiki",
    "beetroot",
    "blacknightshade",
    "white_eggplant",
    "green_eggplant",
    "purple_eggplant",
    "dodo"
]
season=[1,2]
veg_list=[]
for k in veg:
    for j in season:
        vegk=f'sn_{j}_{k}_Market_Price'
        if vegk in data.columns:
            veg_list.append(vegk)

In [ ]:
# list of all seasonal crops ans seasons
crops=['gnuts','beans','irish_potatoes','maize','peas',
      'ginger','garlic','rice','barley','sorghum','millet',
      'soya_beans']
seasons=[1,2]

In [ ]:
# Converting all seasonal crop total_yield and total quantity planted to numeric
for crop in crops:
    for season in seasons:
        total_yield = f'sn_{season}_{crop}_Total_Yield'
        planted = f'sn_{season}_{crop}_planted'
        
        if total_yield not in data.columns:
            data[total_yield] = np.nan
        if planted not in data.columns:
            data[planted] = np.nan
        data[total_yield] = pd.to_numeric(data[total_yield], errors='coerce')
        data[planted] = pd.to_numeric(data[planted], errors='coerce')

In [ ]:
# creating function to compute yield per  unit
def calculate_yield_per_plant(data, crops, seasons):
    result = data.copy()
    
    for crop in crops:
        for season in seasons:
            total_yield = f'sn_{season}_{crop}_Total_Yield'
            planted = f'sn_{season}_{crop}_planted'
            yield_per_plant = f'sn_{season}_{crop}_yp'
            
            # Ensure the yield_per_plant column exists
            if yield_per_plant not in result.columns:
                result[yield_per_plant] = np.nan

            if total_yield in result.columns and planted in result.columns:
                # Identify rows where both total_yield and planted are missing
                null_mask = result[total_yield].isna() & result[planted].isna()
                result.loc[null_mask, yield_per_plant] = np.nan  # Explicitly set to NaN
                
                # Compute yield per plant where valid values exist
                mask = (
                    pd.notna(result[total_yield]) & 
                    pd.notna(result[planted]) & 
                    (result[planted] != 0)
                )
                
                if mask.any():
                    result.loc[mask, yield_per_plant] = (
                        result.loc[mask, total_yield] / result.loc[mask, planted]
                    )
                    
                    # Replace infinite values with NaN
                    result[yield_per_plant].replace([np.inf, -np.inf], np.nan, inplace=True)
    
    return result
data = calculate_yield_per_plant(data, crops, seasons)


In [ ]:
# converting all these columns below to numeric 
required_columns = [
    'Distance_travelled_one_way_OPD_treatment', 'Time_travel_one_way_trip_OPD_treatment_minutes',
    'water_distance_collect_water_round_trip', 'hh_water_collection_Minutes', 'distance_primary_market',
    'time_primary_market', 'Size_land_owned', 'Value_land_owned', "govt_assistance_value", 
    "remittance_gifts_children", "remittance_gifts_relative", "remittance_gifts_friends",
    "casual_work_wage_member1", "casual_work_inkind_member1", "casual_work_wage_member2", 
    "casual_work_inkind_member2", "casual_work_wage_member3", "casual_work_inkind_member3", 
    "casual_work_wage_member4", "casual_work_inkind_member4", "casual_work_wage_member5", 
    "casual_work_inkind_member5", "casual_work_wage_member6", "casual_work_inkind_member6"
]

for col in required_columns:
    if col not in data.columns:
        data[col] = np.nan


data[required_columns] = data[required_columns].astype(float)

In [ ]:
# converting the other columns to numeric
col_price = [i for i in data.columns if "price" in i or "payment_fees_labour" in i or "remittance" in i or "market_ppu" in i or "Price" in i]
data[col_price] = data[col_price].apply(pd.to_numeric, errors='coerce')


In [ ]:
# function computing total number of errors
price_df = price_df.set_index('Crop')
def check_price_violations(row):
    errors = 0  
    for crop in price_df.index:
        if crop in row:  
            if row[crop] < price_df.loc[crop, 'Min'] or row[crop] > price_df.loc[crop, 'Max']:
                errors += 1  
    return errors
data['error'] = data.apply(check_price_violations, axis=1)

In [ ]:
price_df=price_df.reset_index('Crop')

In [ ]:
def highlight_out_of_range(cell_value, min_value, max_value):
    if pd.notna(cell_value) and isinstance(cell_value, (int, float)):  # Ensure it's numeric
        if cell_value < min_value or cell_value > max_value:
            return "background-color: red"
    return ""

def apply_highlighting(data, price_df):
    # Convert price_df to a dictionary for fast lookup (case insensitive)
    crop_prices = {row["Crop"].strip(): (row["Min"], row["Max"]) for _, row in price_df.iterrows()}

    # Apply the highlighting function column-wise
    def highlight_column(column):
        crop_name = column.name.strip()  # Normalize column name
        price_range = crop_prices.get(crop_name)  # Safe dictionary lookup
        if price_range:
            min_price, max_price = price_range
            return column.apply(lambda x: highlight_out_of_range(x, min_price, max_price))
        return [""] * len(column)  # No styling if no matching crop

    return data.style.apply(highlight_column, subset=data.columns)

In [ ]:
col = [i for i in data.columns if "Value_remittance_gift" in i or "Value_payment_fees_labour" in i or 'remittance' in i]

def extract_crop(col_name):
    match = re.search(r'sn_\d+_([a-zA-Z_]+)_Yield|sn_\d+_([a-zA-Z_]+)_Value|([a-zA-Z_]+)_Yield|([a-zA-Z_]+)_Value|([a-zA-Z_]+)_remittance|([a-zA-Z_]+)_payment_fees', col_name)
    if match:
        return match.group(1) or match.group(2) or match.group(3) or match.group(4) or match.group(5) or match.group(6)
    return "Unknown"
df = pd.DataFrame({'Column Name': col, 'Crop Name': [extract_crop(coll) for coll in col]})
data[col] = data[col].apply(pd.to_numeric, errors='coerce')

In [ ]:
business_number = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,97] 
for bus in business_number:
    bus_sa = f'business{bus}_sales'
    bus_po = f'business{bus}_profit'
    if bus_sa not in data.columns and bus_po not in data.columns:
        data[bus_sa]=np.nan
        data[bus_po]=np.nan
    
    if bus_sa in data.columns and bus_po in data.columns:
        data[[bus_sa,bus_po]]=data[[bus_sa,bus_po]].astype(float)
    else:
        print(f"Columns {bus_sa} or {bus_po} do not exist in the dataset.")

In [ ]:
def business_error(row):
    error = 0
    for i in business_number:
        bus_sa = f'business{i}_sales'
        bus_po = f'business{i}_profit'
        if pd.notna(row[bus_po]) and pd.notna(row[bus_sa]):
            if row[bus_po] > row[bus_sa]:
                error += 1
    return error
    

In [ ]:
# def business_error(row):
#     error_bus = 0  

#     for bus in range(1, 29):  # Loop through business numbers from 1 to 28
#         bus_sa = f'business{bus}_sales'
#         bus_po = f'business{bus}_profit'

#         # Use .get() to avoid KeyErrors
#         sales = row.get(bus_sa, None)
#         profit = row.get(bus_po, None)

#         # Ensure both values exist and are numeric before comparison
#         if sales is not None and profit is not None:
#             try:
#                 if float(profit) > float(sales):  
#                     error_bus += 1  
#             except ValueError:
#                 pass  # Ignore non-numeric values

#     return error_bus

In [ ]:
data['error_bus']=data.apply(business_error,axis=1)

In [ ]:
data['error_bus'].min(),data['error_bus'].max()

In [ ]:
def count_remittance_highlights(row):
    highlight_count = 0  

    # Identify relevant columns
    cols = [col for col in row.index if any(x in str(col) for x in 
                ["Value_remittance_gift", "Value_payment_fees_labour", "remittance_gifts_friends"])]

    # Iterate through selected columns
    for col_name in cols:
        if pd.notna(row[col_name]):  
            if row[col_name] < 100 or row[col_name] % 100 != 0:
                highlight_count += 1  

    return highlight_count

In [ ]:
data['remittance_highlight_count'] = data.apply(count_remittance_highlights, axis=1)

In [ ]:
data['remittance_highlight_count'].min(),data['remittance_highlight_count'].max()

In [ ]:
def compute_travel_errors(row):
    error_count = 0  # Initialize error counter

    # Define limits for distance and time
    travel_checks = {
        'Distance_travelled_one_way_OPD_treatment': (0, 28),
        'Time_travel_one_way_trip_OPD_treatment_minutes': (0, 420),
        'water_distance_collect_water_round_trip': (0, 8),
        'hh_water_collection_Minutes': (0, 420),
        'distance_primary_market': (0, 10),
        'time_primary_market': (0, 420)
    }

    # Count errors for extreme values (0 or exceeding max limit)
    for col, (min_val, max_val) in travel_checks.items():
        if col in row:
            if row[col] == min_val or row[col] >= max_val:
                error_count += 1  # Increment error count

    # Walking speed rule: Expected travel time based on distance
    if 'Distance_travelled_one_way_OPD_treatment' in row and 'Time_travel_one_way_trip_OPD_treatment_minutes' in row:
        expected_time_opd = row['Distance_travelled_one_way_OPD_treatment'] * 40
        if row['Time_travel_one_way_trip_OPD_treatment_minutes'] > 1.2 * expected_time_opd:
            error_count += 1

    if 'water_distance_collect_water_round_trip' in row and 'hh_water_collection_Minutes' in row:
        expected_time_water = row['water_distance_collect_water_round_trip'] * 60
        if row['hh_water_collection_Minutes'] > 1.2 * expected_time_water:
            error_count += 1

    if 'distance_primary_market' in row and 'time_primary_market' in row:
        expected_time_market = row['distance_primary_market'] * 40
        if row['time_primary_market'] > 1.2 * expected_time_market:
            error_count += 1

    return error_count  # Return total errors for the row

# Apply function to compute errors
data['travel_errors'] = data.apply(compute_travel_errors, axis=1)


In [ ]:
data['travel_errors'].min(),data['travel_errors'].max()

In [ ]:
def compute_crop_yield_errors(row):
    error_count = 0  # Initialize error counter

    # Define crop yield thresholds
    crop_yield_thresholds = {
        'sn_1_beans_yp': 100, 'sn_2_beans_yp': 100,
        'sn_1_maize_yp': 250, 'sn_2_maize_yp': 250,
        'sn_1_peas_yp': 150, 'sn_2_peas_yp': 150,
        'sn_1_gnuts_yp': 150, 'sn_2_gnuts_yp': 150,
        'sn_1_irish_potatoes_yp': 10, 'sn_2_irish_potatoes_yp': 10,
        'sn_1_ginger_yp': 100, 'sn_2_ginger_yp': 100,
        'sn_1_rice_yp': 150, 'sn_2_rice_yp': 150,
        'sn_1_barley_yp': 100, 'sn_2_barley_yp': 100,
        'sn_1_sorghum_yp': 100, 'sn_2_sorghum_yp': 100,
        'sn_1_millet_yp': 200, 'sn_2_millet_yp': 200,
        'sn_1_soya_beans_yp': 100, 'sn_2_soya_beans_yp': 100,
        'sn_1_garlic_yp': 100, 'sn_2_garlic_yp': 100
    }

    # Check for crop yield exceeding thresholds
    for crop, threshold in crop_yield_thresholds.items():
        if crop in row and row[crop] > threshold:
            error_count += 1  # Increment error count

    return error_count  # Return total errors for the row

# Apply function to compute crop yield errors
data['crop_yield_errors'] = data.apply(compute_crop_yield_errors, axis=1)


In [ ]:
data['crop_yield_errors'].min(),data['crop_yield_errors'].max()

In [ ]:
def compute_land_value_errors(row):
    error_count = 0  # Initialize error counter

    # Land Value Check
    if 'Size_land_owned' in row and 'Value_land_owned' in row:
        expected_land_value = row['Size_land_owned'] * 15_000_000  # Expected value per unit size
        if row['Value_land_owned'] > expected_land_value:
            error_count += 1  # Increment error count

    return error_count  # Return total errors for the row

# Apply function to compute land value errors
data['land_value_errors'] = data.apply(compute_land_value_errors, axis=1)

In [ ]:
data['land_value_errors'].min(),data['land_value_errors'].max()

In [ ]:
def highlight_time(row):
    colors = [''] * len(row)

    # Highlight remittance and payment fee values
    col = [i for i in data.columns if "Value_remittance_gift" in i or "Value_payment_fees_labour" in i or 'remittance_gifts_friends' in i]
    for col_name in col:
        if col_name in row.index and not pd.isna(row[col_name]):
            if row[col_name] < 100 or row[col_name] % 100 != 0:
                colors[row.index.get_loc(col_name)] = 'background-color:red'

    # Sample Check
    if row['sample'] < 6:
        colors[row.index.get_loc('sample')] = 'background-color:red'

    # OPD Travel Checks
    if row['Distance_travelled_one_way_OPD_treatment'] == 0 or row['Distance_travelled_one_way_OPD_treatment'] >= 28:
        colors[row.index.get_loc('Distance_travelled_one_way_OPD_treatment')] = 'background-color:red'

    if row['Time_travel_one_way_trip_OPD_treatment_minutes'] == 0 or row['Time_travel_one_way_trip_OPD_treatment_minutes'] >= 420:
        colors[row.index.get_loc('Time_travel_one_way_trip_OPD_treatment_minutes')] = 'background-color:red'

    # Water Collection Checks
    if row['water_distance_collect_water_round_trip'] == 0 or row['water_distance_collect_water_round_trip'] > 8:
        colors[row.index.get_loc('water_distance_collect_water_round_trip')] = 'background-color:red'

    if row['hh_water_collection_Minutes'] == 0 or row['hh_water_collection_Minutes'] >= 420:
        colors[row.index.get_loc('hh_water_collection_Minutes')] = 'background-color:red'

    # Market Travel Checks
    if row['distance_primary_market'] == 0 or row['distance_primary_market'] > 10:
        colors[row.index.get_loc('distance_primary_market')] = 'background-color:red'

    if row['time_primary_market'] == 0 or row['time_primary_market'] >= 420:
        colors[row.index.get_loc('time_primary_market')] = 'background-color:red'

    # 🚨 Walking Speed Rule Check (Only for high travel time)
    expected_time_opd = row['Distance_travelled_one_way_OPD_treatment'] * 40
    if row['Time_travel_one_way_trip_OPD_treatment_minutes'] > 1.2 * expected_time_opd:
        colors[row.index.get_loc('Distance_travelled_one_way_OPD_treatment')] = 'background-color:red'
        colors[row.index.get_loc('Time_travel_one_way_trip_OPD_treatment_minutes')] = 'background-color:red'

    expected_time_water = row['water_distance_collect_water_round_trip'] * 60
    if row['hh_water_collection_Minutes'] > 1.2 * expected_time_water:
        colors[row.index.get_loc('water_distance_collect_water_round_trip')] = 'background-color:red'
        colors[row.index.get_loc('hh_water_collection_Minutes')] = 'background-color:red'

    expected_time_market = row['distance_primary_market'] * 40
    if row['time_primary_market'] > 1.2 * expected_time_market:
        colors[row.index.get_loc('distance_primary_market')] = 'background-color:red'
        colors[row.index.get_loc('time_primary_market')] = 'background-color:red'

    # Business Check
    business_number = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,97]
    for bus in business_number:
        bus_sa = f'business{bus}_sales'
        bus_po = f'business{bus}_profit'
        if bus_sa in row.index and bus_po in row.index:
            if row[bus_po] > row[bus_sa]:
                colors[row.index.get_loc(bus_po)] = 'background-color:red'
                colors[row.index.get_loc(bus_sa)] = 'background-color:red'

    # Crop Yield Checks
    crop_yield_thresholds = {
        'sn_1_beans_yp': 100, 'sn_2_beans_yp': 100,
        'sn_1_maize_yp': 250, 'sn_2_maize_yp': 250,
        'sn_1_peas_yp': 150, 'sn_2_peas_yp': 150,
        'sn_1_gnuts_yp': 150, 'sn_2_gnuts_yp': 150,
        'sn_1_irish_potatoes_yp': 10, 'sn_2_irish_potatoes_yp': 10,
        'sn_1_ginger_yp': 100, 'sn_2_ginger_yp': 100,
        'sn_1_rice_yp': 150, 'sn_2_rice_yp': 150,
        'sn_1_barley_yp': 100, 'sn_2_barley_yp': 100,
        'sn_1_sorghum_yp': 100, 'sn_2_sorghum_yp': 100,
        'sn_1_millet_yp': 200, 'sn_2_millet_yp': 200,
        'sn_1_soya_beans_yp': 100, 'sn_2_soya_beans_yp': 100,
        'sn_1_garlic_yp': 100, 'sn_2_garlic_yp': 100
    }

    for crop, threshold in crop_yield_thresholds.items():
        if crop in row.index and row[crop] > threshold:
            colors[row.index.get_loc(crop.replace("_yp", "_planted"))] = 'background-color:red'
            colors[row.index.get_loc(crop.replace("_yp", "_Total_Yield"))] = 'background-color:red'
            colors[row.index.get_loc(crop)] = 'background-color:red'

    # Land Value Check
    if 'Size_land_owned' in row.index and 'Value_land_owned' in row.index:
        expected_land_value = row['Size_land_owned'] * 15000000
        if row['Value_land_owned'] > expected_land_value:
            colors[row.index.get_loc('Size_land_owned')] = 'background-color:red'
            colors[row.index.get_loc('Value_land_owned')] = 'background-color:red'

    # Time Checks
    if 'is_start_time_beyond_8pm' in row.index and row['is_start_time_beyond_8pm'] == 1:
        colors[row.index.get_loc('starttime')] = 'background-color:red'

    if 'is_duration_invalid' in row.index and row['is_duration_invalid'] == 1:
        colors[row.index.get_loc('duration2')] = 'background-color:red'

    # Date Difference Check
    if 'date_difference' in row.index and row['date_difference'] > 0:
        colors[row.index.get_loc('starttime')] = 'background-color:red'
        colors[row.index.get_loc('endtime')] = 'background-color:red'

    return colors

In [ ]:
styled=styled_df = apply_highlighting(data, price_df).apply(highlight_time, axis=1)

In [ ]:
excel_output="styled_output2.xlsx"

In [ ]:
pd.set_option("styler.render.max_elements", 2660658)
# st.dataframe(styled)

In [ ]:
styled.to_excel(excel_output, engine="openpyxl", index=False)

In [ ]:
time.sleep(15)

In [ ]:
# recipient_number = "+256752537353"
# kit.sendwhats_image(recipient_number, excel_output, "Here is the analysis result.")

## 